# Map Generation using Celular Automata

It's common to require maps to test your algorithms on. However, a common pitfall is not introducing enough variance in the kinds of maps you experiment with. Humans make terrible random generators, and many rely on placing random ellipses on the screen, despite the fact that nothing in the real world is ever so nice and even. It's also boring.

So we need a technique for generating random maps. Our requirements are:

1. The maps are random.
2. The free space should be relatively contiguous, and so should each obstacle.

## Attempt #1

In [ ]:
# %config InlineBackend.figure_format = 'pdf'
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

import itertools
import functools
import operator
import numpy as np
import matplotlib.pyplot as plt
# Apparently, SNS stands for "Samuel Norman Seaborn", a fictional
# character from The West Wing.
import seaborn as sns

sns.set()

Because we're going to be plotting the maps quite a bit, define a handy helper function to plot them.

In [ ]:
def plot_map(A, **kwargs):
    """Plots the given map with a binary colormap."""
    sns.heatmap(A,
                cbar=False, square=True, xticklabels=False, yticklabels=False,
                # Pick a colormap to draw live cells as black and dead cells as white.
                cmap=sns.cubehelix_palette(dark=0.1, light=0.95),
                **kwargs,
               )

It's easy to generate a "spongy" map. Just decide on the ratio of free space to obstacles

In [ ]:
rows, cols = 20, 20
percentage = 0.40

A = np.zeros(rows * cols, dtype=np.bool)
A[:int(percentage * rows * cols)] = True

plt.title('Map before shuffling')
plot_map(A.reshape(rows, cols))
plt.show()

and shake it around

In [ ]:
np.random.shuffle(A)
# Reshape the array from a 1D array to a 2D matrix
A = np.reshape(A, (rows, cols))

plt.title('Map after shuffling')
plot_map(A)
plt.show()

Lets write this up as a function so we have access to it later. I'm going to call the function `generate_seed()` because I happen to know I'll be using it later.

In [ ]:
def generate_seed(shape, percentage):
    """Generate a randomly filled binary array.
    
    :param shape: The shape of the array to generate.
    :type shape: An int or tuple of ints.
    :param percentage: The percentage of 0s to fill.
    """
    elements = shape
    if isinstance(shape, tuple):
        elements = functools.reduce(operator.mul, shape, 1)
    # Generate a 1D array with the right number of elements.
    A = np.zeros(elements, dtype=np.bool)
    A[:int(percentage * elements)] = True
    np.random.shuffle(A)
    # Return a view of the array with the right dimensions.
    return np.reshape(A, shape)

Note that while this function can generate an dimension of arrays, we'll deal only with the 2D case.

## Attempt #2

After a bit of research, it seems that using Cellular Automata to simulate the array for several time steps as if it were full of live cells that can die and reproduce. This might be able (under the right conditions) turn an unnaturally random array of unrelated cells into a collection of distinct blobs. The key in cellular automata is that there is an inherent relationship between a cell and its neighbors. So let's begin by extracting the neighbors from a given cell.

In [ ]:
def neighbors(A, coord):
    """Get the 3x3 submatrix around the coordinate (i, j).
    
    :param A: The array to extract the submatrix from.
    :type A: A numpy array with shape (M, N).
    """
    i, j = coord
    # Avoid negative indices by clamping down to 0. Positive indices
    # outside the array are fine because slicing will only go up to
    # the edge.
    xmin, ymin = np.clip([i - 1, j - 1], 0, None)
    # End point in a slice is exclusive, so add 2.
    return A[xmin : i + 2, ymin : j + 2]

Now test it to make sure we wrote it correctly.

In [ ]:
A = generate_seed((4, 4), 0.5)
plt.title('neighbor example')
plot_map(A)
plt.show()

In [ ]:
plt.title('Upper left neighbors')
plot_map(neighbors(A, (1, 1)))
plt.show()

In [ ]:
plt.title('Lower right neighbors')
plot_map(neighbors(A, (3, 3)))
plt.show()

Notice how `neighbors()` returns a $2 \times 2$ submatrix rather than a $3 \times 3$ when the point we gave it was in the bottom right corner.

Now we need to define our cellular automata rules for reproduction and death. In one time step,

1. A dead cell becomes alive if it is surrounded by 6 or more living cells.
2. A living cell dies if it is surrounded by 3 or fewer living cells.

Note that the exact conditions are configurable. You might pick different requirements to suite your needs.

So we write a function to run one time step on a given map.

In [ ]:
def timestep(A):
    """Runs a single time step on the given array of cells.
    
    :param A: The 2D numpy array of cells.
    """
    rows, cols = A.shape
    # Iterate over every cell in the array.
    for cell in itertools.product(range(rows), range(cols)):
        # Get the neighbors of the current cell.
        B = neighbors(A, cell)
        # A dead cell becomes alive if surrounded by more than 5 neighbors.
        if not A[cell] and np.sum(B) >= 6:
            A[cell] = True
        # A living cell dies if surrounded by 3 or fewer neighbors.
        elif A[cell] and np.sum(B) <= 3:
            A[cell] = False

In [ ]:
A = generate_seed((20, 20), 0.4)

fig, axes = plt.subplots(2, 2)
axes = np.reshape(axes, 4)

axes[0].set_title('$t = 0$')
plot_map(A, ax=axes[0])

# Run 4 time steps and plot the results.
for i, ax in enumerate(axes[1:]):
    timestep(A)
    ax.set_title(f'$t = {i + 1}$')
    plot_map(A, ax=ax)

plt.show()

Note that not much changes in later iterations. This algorithm converges quickly, for better or for worse. Running the algorithm more times only gets rid of small obstacles, and doesn't really change the large ones.

## Additional Notes

It is common to regenerate with a different random seed if the obstacles do not meet some set of conditions, such as maximum or minimum size, but it's a fast enough algorithm to run multiple times and visually evaluate the quality of the output.

The tweakable parameters are the starting seed, the ratio of alive to dead cells, the reproduction and death rules, and the number of iterations to run.

There might also be a way to inject more random behavior with every iteration, possibly with randomness added to the reproductions and deaths each timestep. I'll avoid a philosophical discussion on the matter.

In [ ]:
A = generate_seed((100, 100), 0.4)
timestep(A)
plot_map(A)